In [1]:
import os
from tqdm import tqdm
import json
import numpy as np

In [2]:
op_root = "/Vol1/dbstore/datasets/k.iskakov/azure_people_test/openpose"
new_op_root = "/Vol1/dbstore/datasets/k.iskakov/azure_people_test/openpose_vakhitov_format"

In [3]:
subject_ids = sorted(os.listdir(op_root))
for subject_id in tqdm(subject_ids):
    camera_ids = sorted(os.listdir(os.path.join(op_root, subject_id)))
    for camera_id in camera_ids:
        op_dir = os.path.join(op_root, subject_id, camera_id)
        
        op_dict = dict()
        keypoint_names = sorted(os.listdir(op_dir))
        for keypoint_name in keypoint_names:
            with open(os.path.join(op_dir, keypoint_name)) as f:
                op = json.load(f)
                
            n_persons = len(op['people'])
            if n_persons == 0:
                continue

            body = np.concatenate([op['people'][i]['pose_keypoints_2d'] for i in range(n_persons)], axis=0).reshape(n_persons, -1, 3)
            face = np.concatenate([op['people'][i]['face_keypoints_2d'] for i in range(n_persons)], axis=0).reshape(n_persons, -1, 3)
            left_hand = np.concatenate([op['people'][i]['hand_left_keypoints_2d'] for i in range(n_persons)], axis=0).reshape(n_persons, -1, 3)
            right_hand = np.concatenate([op['people'][i]['hand_right_keypoints_2d'] for i in range(n_persons)], axis=0).reshape(n_persons, -1, 3)

            item = {
                'body': body.tolist(),
                'face': face.tolist(),
                'left_hand': left_hand.tolist(),
                'right_hand': right_hand.tolist()
            }
            
            op_dict[keypoint_name.replace('_keypoints.json', '')] = item
        
        save_dir = os.path.join(new_op_root, subject_id, camera_id)
        os.makedirs(save_dir, exist_ok=True)
        
        save_path = os.path.join(save_dir, "openpose.json")
        with open(save_path, 'w') as f:
            json.dump(op_dict, f)

100%|██████████| 4/4 [00:12<00:00,  3.20s/it]
